In [1]:
import torch, dgl
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from dgl import DGLGraph
from dgl.data import CoraGraphDataset
from dgl.nn.pytorch import SAGEConv
import dgl.function as fn
from colorama import Fore
from urllib.request import urlretrieve

In [2]:
class SAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """
    
    def __init__(self, in_feat:int, out_feat:int):
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        """Forward computation

        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        """
        with g.local_scope():
            g.ndata['h'] = h
            # update_all is a message passing API.
            g.update_all(message_func = fn.copy_u('h', 'm'), 
                         reduce_func = fn.mean('m', 'h_N'))
            h_N = g.ndata['h_N']
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [3]:
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats)
        self.conv2 = SAGEConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [6]:
dataset = CoraGraphDataset()
g = dataset[0]

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [9]:
def train(g, model:nn.Module, epochs:int, device= torch.device('cpu')):
    model.to(device)
    g.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    all_logits = []
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(1, epochs+1):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that we should only compute the losses of the nodes in the training set,
        # i.e. with train_mask 1.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        all_logits.append(logits.detach())

        if not e % 25:
            print('In epoch {:3d}, loss: {:.4f}, val acc: {:.4f} (best {:.4f}), test acc: {:.4f} (best {:.4f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

In [7]:
model = GraphSAGE(g.ndata['feat'].shape[1], 16, dataset.num_classes)

In [11]:
train(g, model, 100)

In epoch  25, loss: 0.0000, val acc: 0.7620 (best 0.7620), test acc: 0.7620 (best 0.7610)
In epoch  50, loss: 0.0000, val acc: 0.7620 (best 0.7640), test acc: 0.7600 (best 0.7610)
In epoch  75, loss: 0.0000, val acc: 0.7600 (best 0.7640), test acc: 0.7620 (best 0.7610)
In epoch 100, loss: 0.0000, val acc: 0.7600 (best 0.7640), test acc: 0.7610 (best 0.7610)


In [13]:
torch.save(model.state_dict(), 'graphSAGE.pth')